In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

from sklearn.linear_model import LinearRegression
from urllib.request import Request, urlopen


In [ ]:
#Retrieving data from NVE
def get_NVE_data():
    """A function to retrieve data from NVE

    Returns:
        A list with a dictionary containing the results.
    """
    #Hardcoded parameters can be passed as variables.
    api_key = 'VERauKbJ40GNb4FmcthF0Q== '
    station = '12.298.0'
    parameter = '1003'                      #parameter defines the type of data
    resolution_time = '60'                  #resolution in minutes
    reference_time = '2021-01-01/2021-12-31'

    baseurl = "https://hydapi.nve.no/api/v1/Observations?StationId={station}&Parameter={parameter}&ResolutionTime={resolution_time}"

    url = baseurl.format(station=station, parameter=parameter,
                         resolution_time=resolution_time)
    
    #If reference_time parameter is provided, then update the url.
    if reference_time is not None:
        url = "{url}&ReferenceTime={reference_time}".format(
        url=url, reference_time=reference_time)

    print('Retrieving data from:', url)

    request_headers = {
        "Accept": "application/json",
        "X-API-Key": api_key
    }

    request = Request(url, headers=request_headers)
    response = urlopen(request)
    content = response.read().decode('utf-8')
    parsed_result = json.loads(content)
    
    return parsed_result['data']
        

In [ ]:
#Retrievining the data from the local csv-file containing stator temperatures and effects.

data_2021 = pd.read_csv('data_2021.csv')
data_2021['timestamp'] = pd.to_datetime(data_2021['timestamp'], utc=True)
data_2021.set_index('timestamp', inplace=True)   
data_2021
   

In [ ]:
# Reshaping dataframe based on column values in "id"-column and interpolating to hourly resolution
data = data_2021.pivot(columns='id', values='value')
data = data.resample('H').interpolate()
 
data.head()


In [ ]:
NVE_data = get_NVE_data()
NVE_data

In [ ]:
#Retrieving data from NVE
NVE_data = get_NVE_data()


NVE_df = pd.DataFrame(NVE_data[0]['observations'])
NVE_df.drop(['correction', 'quality'], axis=1, inplace = True)  #Dropping uneccesary columns
NVE_df['time'] = pd.to_datetime(NVE_df['time'])
NVE_df.set_index('time', inplace=True)

NVE_df.head()

## Merging NVE data with csv-data

In [ ]:
data['WATER_TERMPERATURE'] = NVE_df['value']

data.dropna(inplace=True)

data.head(5)

In [ ]:
# Defining what values to use as features and what values to predict
# When training the model, we want to use data from 2021-01-01 to 2021-07-01 
# When predicting, we want to use data from 2021-07-02 to 2021-12-31


training_start_date = '2021-01-01'
training_end_date = '2021-07-01'
predicting_start_date = '2021-07-02'
predicting_end_date = '2021-12-31'

# Filter data within the specified datetime range
training_data = data.loc[training_start_date:training_end_date]
testing_data = data.loc[predicting_start_date:predicting_end_date]

# Select features (X) and target variable (Y) from the filtered data
X = training_data[['AKTIV_EFFEKT', 'WATER_TERMPERATURE']]
Y = training_data['STATOR_TEMP_L1']

# Creating the linear model
ML1 = LinearRegression().fit(X, Y)

In [ ]:
#Predicting the values and placing it in a separate column in the data dataframe.
testing_data['Prediction'] = ML1.predict(testing_data[['AKTIV_EFFEKT', 'WATER_TERMPERATURE']])

fig = px.line()

fig.add_trace(
                go.Scatter(
                    x=testing_data.index,
                    y=testing_data['STATOR_TEMP_L1'],
                    mode="lines",
                    name='STATOR_TEMP_L1',
                    line=dict(color="blue"),
                )
            )
fig.add_trace(
                go.Scatter(
                    x=testing_data.index,
                    y=testing_data['Prediction'],
                    mode="lines",
                    name='Prediction',
                    line=dict(color="green"),
                )
            )
fig.show()


In [ ]:
testing_data['Difference'] =  testing_data['STATOR_TEMP_L1'] - testing_data['Prediction']
difference_fig = px.line()
difference_fig.add_trace(
                go.Scatter(
                    x=testing_data.index,
                    y=testing_data['Difference'],
                    mode="lines",
                    name='Difference',
                    line=dict(color="green"),
                )
            )